# 爬虫内容

## puppeteer 

### 环境安装 略

In [ ]:
npm install puppeteer@21.11.0

In [ ]:
npm install puppeteer-extra puppeteer-extra-plugin-stealth


1 docker 不能成功执行

https://pptr.dev/guides/page-interactions

使用的时候,直接复制运行包过去

## 框架(模板)

In [ ]:
%%writefile test1.js
const puppeteer = require('puppeteer-extra');
const StealthPlugin = require('puppeteer-extra-plugin-stealth');
const path = require('path');
const fs = require('fs');

puppeteer.use(StealthPlugin());

function sleep(ms) {
  return new Promise(resolve => setTimeout(resolve, ms));
}

// 从命令行参数中获取参数
const industryText = process.argv[2];
const companyName = process.argv[3];
const slogan = process.argv[4];

(async () => {
    const browser = await puppeteer.launch({
        headless: false, // 非无头模式
        executablePath: '/Users/zhaoxuefeng/.cache/puppeteer/chrome/mac_arm-121.0.6167.85/chrome-mac-arm64/Google Chrome for Testing.app/Contents/MacOS/Google Chrome for Testing',
    });
    
    
    const page = await browser.newPage();

    await page.setViewport({
        width: 1200,
        height: 1200,
        deviceScaleFactor: 1,
    });

    
    await page.goto('https://looka.com/explore', {
        waitUntil: "domcontentloaded",
        timeout: 40000,
    });
    // 运行动作
    
    // 动作结束
    
    await browser.close();
})();


In [ ]:
!node test1.js "industryText" "companyName" "slogan"

## 加载浏览器

In [ ]:
// Launch the browser and open a new blank page
const browser = await puppeteer.launch({
    headless: false, // 非无头模式
    executablePath: '/Users/zhaoxuefeng/.cache/puppeteer/chrome/mac_arm-121.0.6167.85/chrome-mac-arm64/Google Chrome for Testing.app/Contents/MacOS/Google Chrome for Testing',
    userDataDir: './xxxxx',
    timeout: 60000, // 超时时间增加到60秒
    });
    const page = await browser.newPage();

## 设置浏览器大小

In [ ]:
await page.setViewport({
    width: 1200,
    height: 1200,
    deviceScaleFactor: 1,
});


## 跳转网页

In [ ]:
// Navigate the page to a URL
await page.goto('https://kimi.moonshot.cn/');

## 睡眠等待

In [ ]:
await page.waitForTimeout(1000); // 等待页面加载完成

In [ ]:
await page.goto('http://www.baidu.com', {
    waitUntil: "domcontentloaded",
    timeout: timeout,
});

## 设置用户代理?

In [ ]:
await page.setUserAgent('Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36');


## 监听网络请求

In [ ]:
const authHeaders = new Set();

// 监听所有请求
page.on('request', request => {
const headers = request.headers();
if (headers.authorization) {
  console.log(`Authorization found: ${headers.authorization}`);
  authHeaders.add(headers.authorization); // 使用 Set 自动去重
}
});
fs.writeFileSync('authorization_headers.txt', authHeaders.join('\n'));

In [ ]:
const authHeaders = [];

page.on('request', request => {
const headers = request.headers();
if (headers.authorization) {
  console.log(`Authorization found: ${headers.authorization}`);
  authHeaders.push(headers.authorization);
}
});
fs.writeFileSync('authorization_headers.txt', Array.from(authHeaders).join('\n'));


## 定位

In [ ]:
// 方法1 xpath
await page.locator('xpath///*[@id="root"]/div[2]/div/div[2]/div[1]/div/div/div/div[3]/div[1]/div/div/div[2]/div[2]/label');

In [ ]:
// 多元素定位
const tabs = await page.$x(tabSelector);-> [CdpElementHandle]

In [ ]:
# 两者是平替方法
const pp2 = await page.$x("/html/body/div[1]/div[1]/div") // 有括号
const pp = await page.$$("xpath//html/body/div[1]/div[1]/div") // 无括号
    

## 点击

In [ ]:
// 方法2
const element = await page.waitForSelector('::-p-xpath(//*[@id="root"]/div[2]/div/div[2]/div[1]/div/div/div/div[3]/div[1]/div/div/div[2]/div[2]/label)');
await element.click();

## 获取html content

In [ ]:
const htmlContent = await page.content();

In [ ]:
// 特定内容
const htmlContent = await page.$eval('xpath//html/body/main/div[1]/div[3]/div[2]', element => element.innerHTML);

In [ ]:
const tab1HTML = await page.evaluate(element => element.outerHTML, tabs[1]);
console.log(tab1HTML); // 输出 tabs[1] 的 HTML 内容

## 获取html 属性值

In [ ]:
const [element] = await page.$x('//*[@id="main-frame"]/div[1]/div/div/div[2]/div/div/div/div/div[2]/div[2]/div[1]/div/button[2]');
const isDisabled = await page.evaluate(el => el.hasAttribute('disabled'), element);
if (!isDisabled) {

## 局部截图

In [ ]:
const [element] = await page.$x('//*[@id="lg"]');

In [ ]:
#标红 + 截图
if (element) {
    // 截图元素
    await page.evaluate(el => el.style.border = '2px solid red', element);
    await element.screenshot({ path: 'element_screenshot.png' });
    console.log('Screenshot saved as element_screenshot.png');
  } else {
    console.log('No element found for the given XPath.');
  }

In [ ]:
for (let i = 1; i <= 9; i++) {
  console.log(i);
  let eles = `//*[@id="logojoy-root"]/div[2]/div[1]/div[2]/div[2]/div/div/div[${i}]`;
  const [element] = await page.$x(eles);

  if (element) {
    // 截图元素
    // await page.evaluate(el => el.style.border = '4px solid red', element);
    await element.screenshot({ path: `element_screenshot_${i}.png` });
    console.log('Screenshot saved as element_screenshot_i.png');
  } else {
    console.log('No element found for the given XPath.');
  }

}

## 剪切板

In [ ]:
const clipboardData = await page.evaluate(() => {
    return navigator.clipboard.readText();
});

console.log('剪切板内容:', clipboardData);

In [ ]:
const puppeteer = require('puppeteer');

async function getClipboardData() {
    const browser = await puppeteer.launch();
    const page = await browser.newPage();

    // 打开一个包含复制内容的页面示例
    await page.goto('https://example.com');

    // 在页面中执行 JavaScript 读取剪切板内容
    const clipboardData = await page.evaluate(() => {
        return navigator.clipboard.readText();
    });

    console.log('剪切板内容:', clipboardData);

    await browser.close();
}

getClipboardData();

## 文件上传

In [ ]:
await page.goto("https://www.baidu.com");
const soutuBtn = await page.waitForSelector('span.soutu-btn');
//点击上传图片按钮
await soutuBtn.click();
//uploadFile上传图片
const uploadPic = await page.waitForSelector('input.upload-pic');
//上传图片目录自定义
await uploadPic.uploadFile('./aa.png');

窗口弹出式文件上传

In [ ]:
const fileInput = await page.locator('xpath///*[@id="upload_dnd_area"]/div[2]/button');
// fileInput.click();

const [fileChooser] = await Promise.all([
page.waitForFileChooser(), // 等待文件选择框弹出
page.click('xpath///*[@id="upload_dnd_area"]/div[2]/button'),

]);
console.log('1234')
const filePath = path.relative(process.cwd(), './video.mp4');
await fileChooser.accept([filePath]);

## JavaScript 语法

### if

if (a) {} else {}

### 取反

!isDisabledtab

### sleep函数

function sleep(ms) {
  return new Promise(resolve => setTimeout(resolve, ms));
}

await sleep(2000); // 等待2秒

### 箭头函数

region => {...} // 这个是箭头函数 入参是region

### 打日志

console.log('check')

console.log(`success ${api_home}`);

console.log(typeof true);

### 定义变量

const url = process.argv[2];
const timeout = 5000;

const 常量定义

var 变量定义

### 参数传递

In [ ]:
// 获取命令行参数
const args = process.argv.slice(2);
let api_home, url, headless, userdir, executablePath;


args.forEach(arg => {
    const [key, value] = arg.split('=');
    if (key && value) {
        switch (key) {
            case '--api-home':
                api_home = value;
                break;
            case '--url':
                url = value;
                break;
            case '--headless':
                headless = value === 'true';
                break;
            case '--userdir':
                userdir = value;
                break;
            case '--executablePath':
                executablePath = value;
                break;
            default:
                console.log(`Unknown parameter: ${key}`);
        }
    } else {
        console.log(`Invalid parameter format: ${arg}`);
    }
});

// 检查参数是否已传递
if (!api_home || !url || !userdir || !executablePath || headless === undefined) {
    console.error('Missing required parameters. Usage: node workflows.js --api-home=<api-home> --url=<url> --headless=<true|false> --userdir=<userdir> --executablePath=<executablePath>');
    process.exit(1);
}

// 打印参数
console.log(`API Home: ${api_home}`);
console.log(`URL: ${url}`);
console.log(`Headless: ${headless}`);
console.log(`User Directory: ${userdir}`);
console.log(`Executable Path: ${executablePath}`);


### 函数编写

In [ ]:
async function example() {
  console.log('Start');
  await sleep(2000); // 等待2秒
  console.log('2 seconds later');

  const cookies = await page.cookies();
  const fs = require('fs');
  fs.writeFileSync('cookies.json', JSON.stringify(cookies, null, 2));

  console.log('cookie over')
}

In [ ]:
function sleep(ms) {
  return new Promise(resolve => setTimeout(resolve, ms));
}

### 保存文件

In [ ]:
const path = require('path');

In [ ]:
const filePath = path.join(__dirname, 'aa.html');

In [ ]:
const fs = require('fs');

In [ ]:
fs.writeFileSync(filePath, htmlContent, 'utf8');

### 读取文件

In [ ]:
const tasksFile = process.argv[2];
if (!tasksFile) {
    console.error('No tasks file provided');
    process.exit(1);
}

let tasks;
try {
    const data = fs.readFileSync(tasksFile, 'utf8');
    tasks = JSON.parse(data);
} catch (error) {
    console.error('Error reading or parsing tasks file:', error);
    process.exit(1);
}


### 异常抛出

In [ ]:
// 遍历前5个选项卡并点击
for (let i = 0; i < tabs.length && i < 5; i++) {
    console.log(i, nameDict[i]);
    try {
        await tabs[i].click();
        await page.waitForTimeout(1000); // 等待页面加载完成


        const content = await page.content();
        fs.writeFileSync(path.join(api_home, nameDict[i]), content);
    } catch (e) {
        console.error(`Failed to click tab ${i}:`, e);
        fs.writeFileSync(path.join(api_home, nameDict[i]), '');
    }
}

## 多线程 

In [ ]:
%%writefile workflow3.js
const puppeteer = require('puppeteer-extra');
const StealthPlugin = require('puppeteer-extra-plugin-stealth');
const path = require('path');
const fs = require('fs');
const { workerData, parentPort } = require('worker_threads');
puppeteer.use(StealthPlugin());

// const url = 'https://rapidapi.com/Glavier/api/ahrefs1/playground/apiendpoint_b535d225-dfd9-4b62-b1fb-d72e7368aaaf';
// const api_home = 'aaaa';
const headless = false;
// const userdir = './xxxxx';
const executablePath = '/Users/zhaoxuefeng/.cache/puppeteer/chrome/mac_arm-121.0.6167.85/chrome-mac-arm64/Google Chrome for Testing.app/Contents/MacOS/Google Chrome for Testing';

(async () => {
    const { api_home, url,userdir} = workerData;
    // 启动浏览器
    const browser = await puppeteer.launch({
      headless: headless, // 非无头模式
    executablePath: executablePath,
    userDataDir: userdir,
    });

    
    
    const page = await browser.newPage();
    await page.setViewport({
        width: 1200,
        height: 1200,
        deviceScaleFactor: 1,
    });
    
    // 导航到目标页面
    await page.goto(url);

    // 定义选项卡选择器和文件名字典
    const tabSelector = '//*[@id="main-frame"]/div[1]/div/div/div[2]/div/div/div/div/div[1]/div/div/button';
    const nameDict = {
        0: "app.html",
        1: "params.html",
        2: "headers.html",
        3: "body.html",
        4: "authorizations.html"
    };

    // 查找所有选项卡
    const tabs = await page.$x(tabSelector);
    console.log(tabs)
    
    // 遍历前5个选项卡并点击
    for (let i = 0; i < tabs.length && i < 5; i++) {
        console.log(i, nameDict[i]);
        
        await tabs[i].click();
        await page.waitForTimeout(100); // 等待页面加载完成

        const content = await page.content();
        fs.writeFileSync(path.join(api_home, nameDict[i]), content);
    }
    
    // spider_Code Snippets python
    console.log('Snippets python');
    await page.locator('xpath///*[@id="main-frame"]/div[1]/div/div/div[2]/div/div/div/div/div[2]/div[2]/div[1]/div/button[1]').click();
    await page.locator('xpath///*[@id="main-frame"]/div[1]/div/div/div[2]/div/div/div/div/div[2]/div[2]/div[2]/div[1]/div/div/div[1]/div/div').click();
    await page.locator('xpath///*[@id="portal-root"]/div/div/button[14]').click();
    await page.locator('xpath///*[@id="main-frame"]/div[1]/div/div/div[2]/div/div/div/div/div[2]/div[2]/div[2]/div[1]/div/div/div[2]/div/div').click();
    await page.locator('xpath///*[@id="portal-root"]/div/div/button[2]').click();
    
    var content = await page.content();
    fs.writeFileSync(path.join(api_home,'code_snippets_python.html'), content);
    
    // spider_Code Snippets shell
    console.log('Snippets shell');
    await page.locator('xpath///*[@id="main-frame"]/div[1]/div/div/div[2]/div/div/div/div/div[2]/div[2]/div[2]/div[1]/div/div/div[1]/div/div').click();
    await page.locator('xpath///*[@id="portal-root"]/div/div/button[17]').click();
    await page.locator('xpath///*[@id="main-frame"]/div[1]/div/div/div[2]/div/div/div/div/div[2]/div[2]/div[2]/div[1]/div/div/div[2]/div/div').click();
    await page.locator('xpath///*[@id="portal-root"]/div/div/button[1]').click();
    
    var content = await page.content();
    fs.writeFileSync(path.join(api_home,'code_snippets_shell.html'), content);
    
    
    // spider_Example_Response
    console.log('Example_Response');
    await page.locator('xpath///*[@id="main-frame"]/div[1]/div/div/div[2]/div/div/div/div/div[2]/div[2]/div[1]/div/button[2]').click();
    var content = await page.content();
    fs.writeFileSync(path.join(api_home,'Example_Response.html'), content);
    
    

    // 关闭浏览器
    console.log('close');
    await browser.close();
    console.log(`success ${api_home}`);

})();

In [ ]:
%%writefile main.js
const { Worker } = require('worker_threads');
const path = require('path');
const fs = require('fs');




const tasks = [
    {
        api_home: 'output1',
        url: 'https://rapidapi.com/VanitySoft/api/boundaries-io-1/playground/apiendpoint_9fefa91b-1881-43d2-b884-6174f55c0fe9',
        userdir: 'xxxxx',
    },
    {
        api_home: 'output2',
        url: 'https://rapidapi.com/Glavier/api/ahrefs1/playground/apiendpoint_b535d225-dfd9-4b62-b1fb-d72e7368aaaf',
        userdir: 'xxxxx2',
    },
    
    {
        api_home: 'output3',
        url: 'https://rapidapi.com/VanitySoft/api/boundaries-io-1/playground/apiendpoint_9fefa91b-1881-43d2-b884-6174f55c0fe9',
        userdir: 'xxxxx3',
    },
    
    {
        api_home: 'output4',
        url: 'https://rapidapi.com/Glavier/api/ahrefs1/playground/apiendpoint_b535d225-dfd9-4b62-b1fb-d72e7368aaaf',
        userdir: 'xxxxx4',
    },
    
        {
        api_home: 'output5',
        url: 'https://rapidapi.com/Glavier/api/ahrefs1/playground/apiendpoint_b535d225-dfd9-4b62-b1fb-d72e7368aaaf',
        userdir: 'xxxxx5',
    },
    // 可以添加更多任务
];


// 函数用于创建和启动 Worker
function runWorker(task) {
    return new Promise((resolve, reject) => {
        const worker = new Worker(path.resolve(__dirname, 'workflow3.js'), {
            workerData: task
        });

        worker.on('message', (message) => {
            console.log(message);
        });

        worker.on('error', (error) => {
            console.error('Worker error:', error);
            reject(error);
        });

        worker.on('exit', (code) => {
            if (code !== 0) {
                console.error(`Worker stopped with exit code ${code}`);
                reject(new Error(`Worker stopped with exit code ${code}`));
            } else {
                resolve();
            }
        });
    });
}

// 主函数来运行所有的任务
async function main() {
    // 确保任务目录存在
    tasks.forEach(task => {
        if (!fs.existsSync(task.api_home)) {
            fs.mkdirSync(task.api_home);
        }
    });

    // 运行所有的 Worker
    try {
        await Promise.all(tasks.map(runWorker));
        console.log('All tasks completed successfully');
    } catch (error) {
        console.error('Error in tasks:', error);
    }
}

if (require.main === module) {
    main();
}


## 多进程

In [ ]:
 注意 多进程中 没有 workerData 模式 (在多线程中出现)

In [ ]:
%%writefile workflow4.js
const puppeteer = require('puppeteer-extra');
const StealthPlugin = require('puppeteer-extra-plugin-stealth');
const path = require('path');
const fs = require('fs');
// const { workerData, parentPort } = require('worker_threads');


// 解析命令行参数
const args = process.argv.slice(2);
const params = {};
args.forEach(arg => {
    const [key, value] = arg.split('=');
    if (key && value) {
        params[key] = value;
    }
});

const { api_home, url, userdir } = params;

puppeteer.use(StealthPlugin());

// const url = 'https://rapidapi.com/Glavier/api/ahrefs1/playground/apiendpoint_b535d225-dfd9-4b62-b1fb-d72e7368aaaf';
// const api_home = 'aaaa';
const headless = false;
// const userdir = './xxxxx';
const executablePath = '/Users/zhaoxuefeng/.cache/puppeteer/chrome/mac_arm-121.0.6167.85/chrome-mac-arm64/Google Chrome for Testing.app/Contents/MacOS/Google Chrome for Testing';




(async () => {
    // const { api_home, url,userdir} = workerData;
    // 启动浏览器
    const browser = await puppeteer.launch({
      headless: headless, // 非无头模式
    executablePath: executablePath,
    userDataDir: userdir,
    });

    
    
    const page = await browser.newPage();
    await page.setViewport({
        width: 1200,
        height: 1200,
        deviceScaleFactor: 1,
    });
    
    // 导航到目标页面
    await page.goto(url);

    // 定义选项卡选择器和文件名字典
    const tabSelector = '//*[@id="main-frame"]/div[1]/div/div/div[2]/div/div/div/div/div[1]/div/div/button';
    const nameDict = {
        0: "app.html",
        1: "params.html",
        2: "headers.html",
        3: "body.html",
        4: "authorizations.html"
    };

    // 查找所有选项卡
    const tabs = await page.$x(tabSelector);
    console.log(tabs)
    
    // 遍历前5个选项卡并点击
    for (let i = 0; i < tabs.length && i < 5; i++) {
        console.log(i, nameDict[i]);
        
        await tabs[i].click();
        await page.waitForTimeout(100); // 等待页面加载完成

        const content = await page.content();
        fs.writeFileSync(path.join(api_home, nameDict[i]), content);
    }
    
    // spider_Code Snippets python
    console.log('Snippets python');
    await page.locator('xpath///*[@id="main-frame"]/div[1]/div/div/div[2]/div/div/div/div/div[2]/div[2]/div[1]/div/button[1]').click();
    await page.locator('xpath///*[@id="main-frame"]/div[1]/div/div/div[2]/div/div/div/div/div[2]/div[2]/div[2]/div[1]/div/div/div[1]/div/div').click();
    await page.locator('xpath///*[@id="portal-root"]/div/div/button[14]').click();
    await page.locator('xpath///*[@id="main-frame"]/div[1]/div/div/div[2]/div/div/div/div/div[2]/div[2]/div[2]/div[1]/div/div/div[2]/div/div').click();
    await page.locator('xpath///*[@id="portal-root"]/div/div/button[2]').click();
    
    var content = await page.content();
    fs.writeFileSync(path.join(api_home,'code_snippets_python.html'), content);
    
    // spider_Code Snippets shell
    console.log('Snippets shell');
    await page.locator('xpath///*[@id="main-frame"]/div[1]/div/div/div[2]/div/div/div/div/div[2]/div[2]/div[2]/div[1]/div/div/div[1]/div/div').click();
    await page.locator('xpath///*[@id="portal-root"]/div/div/button[17]').click();
    await page.locator('xpath///*[@id="main-frame"]/div[1]/div/div/div[2]/div/div/div/div/div[2]/div[2]/div[2]/div[1]/div/div/div[2]/div/div').click();
    await page.locator('xpath///*[@id="portal-root"]/div/div/button[1]').click();
    
    var content = await page.content();
    fs.writeFileSync(path.join(api_home,'code_snippets_shell.html'), content);
    
    
    // spider_Example_Response
    console.log('Example_Response');
    await page.locator('xpath///*[@id="main-frame"]/div[1]/div/div/div[2]/div/div/div/div/div[2]/div[2]/div[1]/div/button[2]').click();
    var content = await page.content();
    fs.writeFileSync(path.join(api_home,'Example_Response.html'), content);
    
    

    // 关闭浏览器
    console.log('close');
    await browser.close();
    console.log(`success ${api_home}`);

})();


In [ ]:
%%writefile main2.js
const { exec } = require('child_process');
const path = require('path');
const fs = require('fs');

// 定义任务列表
const tasks = [
    {
        api_home: 'output1',
        url: 'https://rapidapi.com/VanitySoft/api/boundaries-io-1/playground/apiendpoint_9fefa91b-1881-43d2-b884-6174f55c0fe9',
        userdir: 'xxxxx',
    },
    {
        api_home: 'output2',
        url: 'https://rapidapi.com/Glavier/api/ahrefs1/playground/apiendpoint_b535d225-dfd9-4b62-b1fb-d72e7368aaaf',
        userdir: 'xxxxx2',
    },
    
    {
        api_home: 'output3',
        url: 'https://rapidapi.com/VanitySoft/api/boundaries-io-1/playground/apiendpoint_9fefa91b-1881-43d2-b884-6174f55c0fe9',
        userdir: 'xxxxx3',
    },
    
    {
        api_home: 'output4',
        url: 'https://rapidapi.com/Glavier/api/ahrefs1/playground/apiendpoint_b535d225-dfd9-4b62-b1fb-d72e7368aaaf',
        userdir: 'xxxxx4',
    },
    
        {
        api_home: 'output5',
        url: 'https://rapidapi.com/Glavier/api/ahrefs1/playground/apiendpoint_b535d225-dfd9-4b62-b1fb-d72e7368aaaf',
        userdir: 'xxxxx5',
    },
    // 可以添加更多任务
];

// 函数用于创建和启动子进程
function runTask(task) {
    return new Promise((resolve, reject) => {
        const command = `node ${path.resolve(__dirname, 'workflow4.js')} ` +
                        `api_home=${task.api_home} ` +
                        `url=${task.url} ` +
                        `headless=${task.headless} ` +
                        `userdir=${task.userdir} ` +
                        `executablePath=${task.executablePath}`;

        const child = exec(command, (error, stdout, stderr) => {
            if (error) {
                console.error(`Error executing task for ${task.api_home}:`, error);
                reject(error);
                return;
            }
            if (stderr) {
                console.error(`stderr for ${task.api_home}:`, stderr);
            }
            console.log(`stdout for ${task.api_home}:`, stdout);
            resolve();
        });
    });
}

// 主函数来运行所有的任务
async function main() {
    // 确保任务目录存在
    tasks.forEach(task => {
        if (!fs.existsSync(task.api_home)) {
            fs.mkdirSync(task.api_home);
        }
    });

    // 运行所有的任务
    try {
        await Promise.all(tasks.map(runTask));
        console.log('All tasks completed successfully');
    } catch (error) {
        console.error('Error in tasks:', error);
    }
}

if (require.main === module) {
    main();
}


# selenium

## 安装selenium chrome 以及chromedriver

%pip install selenium

%pip install chromedriver-py

In [ ]:
https://pypi.org/project/chromedriver-py/#description

In [ ]:
chrome://settings/help  # 查看chrome版本

In [ ]:
https://google-chrome.en.uptodown.com/mac/post-download/102925850  # 历史chrome 版本下载

## 测试环境

In [ ]:
from selenium import webdriver

from chromedriver_py import binary_path

binary_path

svc = webdriver.ChromeService(executable_path=binary_path)
driver = webdriver.Chrome(service=svc)
driver.get("http://www.python.org")

assert "Python" in driver.title

In [ ]:
from selenium import webdriver

from chromedriver_py import binary_path

import time
from selenium.webdriver.common.by import By

svc = webdriver.ChromeService(executable_path=binary_path)
driver = webdriver.Chrome(service=svc)

driver.get("https://www.baidu.com/")
driver.find_element('xpath','//*[@id="kw"]').send_keys('赵丽颖')

driver.find_element(by='xpath',value='//*[@id="su"]').click()
# driver.find_element(by='xpath')
# time.sleep(5)
# driver.quit()

In [ ]:
driver.get(url=url)
driver.quit()  关闭浏览器
driver.close()  关闭当前页
driver.page_source  html 结构源码
driver.page_source.find('字符串')  html源码中搜索指定字符串 没有找到就返回-1 常用来判断是否为最后一页
driver.maximize_window()   浏览器窗口最大化


In [ ]:
find 是字符串的方法 返回下标索引

### 无头

In [ ]:
from selenium import webdriver
from chromedriver_py import binary_path

import time
from selenium.webdriver.common.by import By

svc = webdriver.ChromeService(executable_path=binary_path)

options = webdriver.ChromeOptions()
# 添加无头模式
options.add_argument('--headless')

# driver = webdriver.Chrome(service=svc)
driver = webdriver.Chrome(service=svc,options=options)

## 鼠标移动

In [ ]:
from selenium import webdriver
from selenium.webdriver import ActionChains

ActionChains(driver).move_to_element(element).perform()

# driver.find_element

## 等待元素

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
driver.get(api_url)
print('start')

element = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, '//*[@id="TEST_ENDPOINT_FORM"]/div/div[1]/div/div/span/div'))  # 根据实际情况替换选择器
)

print('end')
# do

## 使用用户的存储数据

In [1]:
from selenium import webdriver

from chromedriver_py import binary_path
from selenium.webdriver.common.by import By
import time

import os

In [2]:
options = webdriver.ChromeOptions()

In [3]:
options.add_argument("user-data-dir=/Users/zhaoxuefeng/Library/Application Support/Google/Chrome")  # 替换为你的 Chrome 用户数据目录路径

In [4]:
svc = webdriver.ChromeService(executable_path=binary_path)

In [5]:
driver = webdriver.Chrome(service=svc,options=options)

In [6]:
driver.get("https://kimi.moonshot.cn/chat/cq6ecd2lnl99t48jr2vg")

## 常规爬虫分析

In [ ]:
""" 向京东官网(https://www.jd.com/)发请求,获取响应内容""" 
import requests  
resp = requests.get(url='https://www.jd.com/') 
# 1.text属性: 获取响应内容-字符串
html = resp.text 
print(html)

# BeautifulSoup

In [ ]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

def get_all_links(url):
    response = requests.get(url)
    if response.status_code != 200:
        print(f"Failed to retrieve the page: {url}")
        return []

    soup = BeautifulSoup(response.content, "html.parser")

    # Finding all 'a' tags which typically contain href attribute for links
    links = [
        (a.text, urljoin(url, a["href"]))
        for a in soup.find_all("a", href=True)
        if a["href"]
    ]

    return links


import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

def get_content(url):
    response = requests.get(url)
    if response.status_code != 200:
        print(f"Failed to retrieve the page: {url}")
        return []

    soup = BeautifulSoup(response.content, "html.parser")
    text_content = soup.get_text()

    return text_content

In [ ]:
%%writefile /Users/zhaoxuefeng/GitHub/zxftools/zxftools/tnode/lib/tools/web.py
from .base import BaseTools
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

class WebTools(BaseTools):
    def __init__(self, register=[]):
        register_default = ['get_all_links', 'get_content']
        super().__init__(register, register_default)

    def get_all_links(self, url):
        """Retrieve all links from a given URL and return a list of tuples containing the link text and the absolute URL"""
        response = requests.get(url)
        if response.status_code != 200:
            print(f"Failed to retrieve the page: {url}")
            return []

        soup = BeautifulSoup(response.content, "html.parser")

        links = [
            (a.text, urljoin(url, a["href"]))
            for a in soup.find_all("a", href=True)
            if a["href"]
        ]

        return links

    def get_content(self, url):
        """Retrieve the text content from a given URL and return it as a string"""
        response = requests.get(url)
        if response.status_code != 200:
            print(f"Failed to retrieve the page: {url}")
            return []

        soup = BeautifulSoup(response.content, "html.parser")
        text_content = soup.get_text()

        return text_content

# pyppeteer

In [ ]:
 上传文件

In [ ]:
import asyncio
from pyppeteer import launch
userdir = 'user3'
async def upload_file():
    #browser = await launch(headless=False)  # 启动浏览器
    browser = await launch(
        headless=False,  # 非无头模式
        userDataDir=userdir
    )
    page = await browser.newPage()
    await page.goto('https://www.yinziai.com/tools/extract-text')  # 访问包含文件上传的页面

    # 等待文件输入框加载完成
    input_element = await page.waitForSelector('input[type="file"]')

    # 设置文件路径以上传文件
    await input_element.uploadFile('/Users/zhaoxuefeng/Documents/Project/Work/video.mp4')  # 替换为你的文件路径

    # 提交表单或进行其他操作
    #await page.click('input[type="submit"]')

    # 等待一些时间以查看结果
    await page.waitFor(20000)

    await browser.close()

asyncio.get_event_loop().run_until_complete(upload_file())


In [ ]:
"""案例"""
import asyncio
from pyppeteer import launch

width, height = 1500, 1300  # 尺寸配置
"""如果要单独写到配置高宽，需要在window-size前面加上f哦"""

async def main():
    browser = await launch(
        headless=True,
        args=['--disable-infobars', f'--window-size={width},{height}'],
    )
    page = await browser.newPage()
    await page.goto('http://www.taobao.com')
    await page.screenshot({'path': 'example.png'})
    #await browser.close()

asyncio.get_event_loop().run_until_complete(main())

In [ ]:
img_element = await page.waitForXPath('/html/body/div[3]/div/div/div/div[2]/div/div/div[1]/div/div[2]/img', timeout=5000)
    img_src = await page.evaluate('(element) => element.src', img_element)
    print(f"图片链接: {img_src}")

In [1]:
### 1 <class 'pyppeteer.element_handle.ElementHandle'>

In [ ]:
elements = await page.xpath('//*[@id="index"]/nav/a/div/a')

In [ ]:
elements[0]

In [ ]:
img_element = await page.waitForXPath('/html/body/div[3]/div/div/div/div[2]/div/div/div[1]/div/div[2]/img', timeout=5000)

In [ ]:
能力 :
.click()


In [ ]:
# page

In [ ]:
page = await browser.newPage()

In [ ]:
await page.setViewport({
        "width": 1200,
        "height": 1200,
        "deviceScaleFactor": 1,
    }) # 设置窗口大小

In [ ]:
await page.goto('https://www.yinziai.com/tools/extract-text')  # 访问包含文件上传的页面
    

In [ ]:
await page.waitFor(1000)  # 等待几秒以查看效果

In [ ]:
#     auth_headers = set()

#     # 监听所有请求
#     async def intercept_request(request):
#         headers = request.headers
#         if 'authorization' in headers:
#             print(f"Authorization found: {headers['authorization']}")
#             auth_headers.add(headers['authorization'])  # 使用 Set 自动去重
#         await request.continue_()

#     await page.setRequestInterception(True)
#     page.on('request', intercept_request)


In [ ]:
    
#     # 定义拦截请求的回调函数
#     async def log_request(request):
#         headers = request.headers
#         if 'authorization' in headers:
#             print(f"Authorization: {headers['authorization']}")
#         await request.continue_()

#     # 启用请求拦截
#     await page.setRequestInterception(True)
    
#     # 使用事件循环的方式来添加监听器，确保异步调用
#     ee = EventEmitter()
#     ee.on('request', lambda req: asyncio.ensure_future(log_request(req)))
    
#     page.on('request', ee.emit)

In [ ]:


    Authorization = ''
    
    async def log_request(request):
        headers = request.headers
        if 'authorization' in headers:
            Authorization = headers['authorization']
            print(f"Authorization: {headers['authorization']}")
        # 立即继续请求，不阻塞程序
        await request.continue_()

    # 启用请求拦截
    await page.setRequestInterception(True)
    
    # 使用 lambda 异步调用 log_request，确保程序继续执行
    page.on('request', lambda req: asyncio.create_task(log_request(req)))
    
    
    
    await page.goto('https://www.yinziai.com/tools/extract-text')  # 访问包含文件上传的页面

    await page.setRequestInterception(False)
    